# Initialization Routine


In order to compute the dynamics in SEOBNRv5HM, we will need accurate quasi-circular initial conditions. these initial conditions need to be computed given the following waveform parameters:

- The total mass $M$ in solar masses
- The mass ratio $q$ given as $\tfrac{m_1}{m_2}$ s.t $q \geq 1$.
- The spins $S_1$, and $S_2$ of the binary in dimensionless units
- The starting gravitational wave frequency in Hz

First, we implement the systems of equations needed to be solved for the primitive variables and then build the initial conditions routine to get the primitive variables from the above inputs.

Additionally, SEOBNRv5HM calculates quantities related to the end stages of the inspiral. These include:

- The stopping radius at which to terminate the integration
- A fit for the strain and frequency of the 

In [1]:
import sys,os#Add sys to get cmdline_helper from NRPy top directory; remove this line and next when debugged
sys.path.append('../')
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface

# Create C code output directory:
Ccodesdir = "Dynamics"
# Then create an output directory in case it does not exist
cmd.mkdir(Ccodesdir)

# Step : Transform mass ratio to individual masses

We use the normalization choice $m_1 + m_2 \equiv M = 1$ to get

$$
m_1 = \frac{q}{1 + q}\\
m_2 = \frac{1}{1 + q}
$$

In [2]:
%%writefile $Ccodesdir/v5HM_BOB_Initial_Conditions_Calibration.txt
m1 = q/(1 + q)
m2 = 1/(1 + q)
nu = m1*m2

Overwriting Dynamics/v5HM_BOB_Initial_Conditions_Calibration.txt


# Step : Incorporate the spins

We define the spins as 

$$
\chi_{1,2} = S_{1,2}
$$

Since in the spin-aligned model only the z-component is needed.

In [3]:
%%writefile -a $Ccodesdir/v5HM_BOB_Initial_Conditions_Calibration.txt
chi1 = S1
chi2 = S2

Appending to Dynamics/v5HM_BOB_Initial_Conditions_Calibration.txt


# Step : Convert the starting frequency to orbital frequency

Noting that the plane of the polarization of the gravitational wave "rotates at twice the orbital rate" (see the "Effects of passing" section of [this Wikipedia article](https://en.wikipedia.org/wiki/Gravitational_wave#Effects_of_passing)), the initial orbital frequency is
 
\begin{equation*}
    \omega = M \mathcal{M}_{\odot} \pi f.
\end{equation*}


In [4]:
%%writefile -a $Ccodesdir/v5HM_BOB_Initial_Conditions_Calibration.txt
Msol = 4.925491025543575903411922162094833998e-6
Omega_0 = M*Msol*np.pi*f

Appending to Dynamics/v5HM_BOB_Initial_Conditions_Calibration.txt


# Step : Compute initial step size

In this step we will define the initial choice for the step size given in line 117 of pyseobnr's [integrate_ode.py](https://git.ligo.org/waveforms/software/pyseobnr/-/blob/main/pyseobnr/eob/dynamics/integrate_ode.py):

$$
h = \frac{2\pi}{5\Omega_0 }
$$

In [5]:
%%writefile -a $Ccodesdir/v5HM_BOB_Initial_Conditions_Calibration.txt

h = 2*np.pi/5/Omega_0

Appending to Dynamics/v5HM_BOB_Initial_Conditions_Calibration.txt


# Step : Get the initial guess for the radius and angular momentum

We use Kepler's law for circular orbits to get the following guess

$$
r_{\rm guess} = \omega^{-2/3}\\
p_{\phi,\rm guess} = r_{\rm guess}^{2}\omega = \omega^{-1/3}
$$

In [6]:
%%writefile -a $Ccodesdir/v5HM_BOB_Initial_Conditions_Calibration.txt
r_guess = Omega_0**(-2/3)
pphi_guess = Omega_0**(-1/3)

Appending to Dynamics/v5HM_BOB_Initial_Conditions_Calibration.txt


# Step : Compute the conservative initial conditions

In this step, we compute the purely conservative (i.e in the absence of radiation reaction) circular initial conditions.
The equations for a circular orbit with orbital frequency $\omega$ is given by:

$$
\dot{\phi} = \omega = \left.\frac{\partial H}{\partial p_{\phi}}\right|_{p_{r_*} = 0}\\
\dot{p}_{r} = -\left.\frac{\partial H}{\partial r}\right|_{p_{r_*} = 0}
$$

We solve the above system of non-linear system of equations.
Note: We are using a tolerance of `6e-12` as this is the choice pyseobnr makes.

In [7]:
%%writefile -a $Ccodesdir/v5HM_BOB_Initial_Conditions_Calibration.txt
sol_cons_guess = np.array([r_guess,pphi_guess])
params_cons = np.array([m1,m2,chi1,chi2,a6,dSO,Omega_0])
sol_cons = root(IC_cons,sol_cons_guess,args = params_cons,tol = 6e-12)
r , pphi = sol_cons.x[0] , sol_cons.x[1]

Appending to Dynamics/v5HM_BOB_Initial_Conditions_Calibration.txt


# Step : Compute the dissipative initial conditions

In this step, we find the value of $p_{r_*}$ using the ansatz:

$$
\frac{d r}{dt} \equiv \left[ \frac{ \frac{d p_{\phi}}{d t} }{ \frac{d p_{\phi}}{d r} } \right].
$$

Where, $\frac{d p_{\phi}}{d t} = \mathcal{F}_{\phi}$. In the quasi-circular limit $d\left(\frac{\partial H}{\partial r}\right) = 0$ we get:

$$
\frac{dl}{dr} = -\frac{ \frac{\partial^2 H}{\partial r^2} }{ \frac{\partial^2 H}{\partial r \partial p_{\phi}} }.
$$

Thus, we can find the initial value by $p_{r_*}$ by solving the following non-linear equation:

$$
\xi\frac{\partial H}{\partial p_{r_{*}}} = -\frac{ \mathcal{F}_{\phi} \frac{\partial^2 H}{\partial r \partial p_{\phi}} }{ \frac{\partial^2 H}{\partial r^2} }
$$

We use a guess bracket (i.e an interval the contains the root c.f [scipy.optimise documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.root_scalar.html#scipy.optimize.root_scalar)) of $[-3\times10^{-2},0]$ that is consistent with pyseobnr. Additionally, pyseobnr uses an absolute tolerance of `1e-12`, and a relative tolerance of `1e-10`.


In [8]:
%%writefile -a $Ccodesdir/v5HM_BOB_Initial_Conditions_Calibration.txt
params_diss = np.array([m1,m2,r,pphi,chi1,chi2,a6,dSO])
prstar_bracket = [-3e-2,0]
prstar_sol = root_scalar(IC_diss, args = params_diss, bracket = prstar_bracket, xtol = 1e-12, rtol = 1e-10)
prstar = prstar_sol.root

Appending to Dynamics/v5HM_BOB_Initial_Conditions_Calibration.txt


# Step : $r_{\rm ISCO}^{\rm eff}$

To define the Kerr ISCO, we also require the spin parameter $a_{f}$ of the final Kerr black hole. In this case, SEOBNRv5 uses the numerical estimate provided by Equations 12-13 of [Hoffman, et. al.](https://iopscience.iop.org/article/10.3847/2041-8205/825/2/L19/pdf), hereby HBR2016. to define this, we first require an effective ISCO radius given by Equation 4-6 of HBR2016:
$$
r_{\rm ISCO}^{\rm eff} = 3 + Z_2 - \frac{a^{\rm eff}}{|a^{\rm eff}|}\sqrt{(3 - Z_1)(3 + Z_1 + 2Z_2)}\\
Z_2 = \sqrt{3 a^{\textrm{eff},2} + Z_1^2}\\
Z_1 = 1 + (1 - a^{\textrm{eff},2})^{1/3}\left[ (1 + a^{\rm eff})^{1/3} + (1 - a^{\rm eff})^{1/3} \right]\\
a^{\rm eff} = a_{\rm tot} + \xi\nu(\chi_1 + \chi_2)\\
a_{\rm tot} = m_1^2\chi_1 + m_2^2\chi_2\\
\xi = 0.474046
$$

Where, the value of $\xi$ is taken from Table 1 of HBR2016 with the choice of $n_M = 3, n_J = 4$ as evident in line 130 of [postadiabatic_C.pyx](https://git.ligo.org/waveforms/software/pyseobnr/-/blob/main/pyseobnr/eob/dynamics/postadiabatic_C.pyx) of the pyseobnr code.

In [9]:
%%writefile -a $Ccodesdir/v5HM_BOB_Initial_Conditions_Calibration.txt
atot = m1*m1*chi1 + m2*m2*chi2
aeff = atot + .474046*nu*(chi1 + chi2)
aeff_max1 = min(aeff,1)
Z1eff = 1 + (np.cbrt(1 - aeff_max1*aeff_max1))*(np.cbrt(1 + aeff_max1) + np.cbrt(1 - aeff_max1))
Z2eff = np.sqrt(3*aeff_max1*aeff_max1 + Z1eff*Z1eff)
rISCOeff = 3 + Z2eff - np.sign(aeff_max1)*np.sqrt((3 - Z1eff)*(3 + Z1eff + Z2eff))

Appending to Dynamics/v5HM_BOB_Initial_Conditions_Calibration.txt


# Step : $L_{\rm ISCO}^{\rm eff}$

Following Equations 2-3 of [HBR2016](https://iopscience.iop.org/article/10.3847/2041-8205/825/2/L19/pdf), we define the effective angular momentum:
$$
L_{\rm ISCO}^{\rm eff} = \frac{2}{3\sqrt{3}}\left[ 1 + 2\sqrt{3r_{\rm ISCO}^{\rm eff} - 2} \right]
$$

In [10]:
%%writefile -a $Ccodesdir/v5HM_BOB_Initial_Conditions_Calibration.txt
LISCOeff = (2/3/np.sqrt(3))*(1 + 2*np.sqrt(3*rISCOeff - 2))

Appending to Dynamics/v5HM_BOB_Initial_Conditions_Calibration.txt


# Step : $E_{\rm ISCO}^{\rm eff}$

Following Equations 2-3 of [HBR2016](https://iopscience.iop.org/article/10.3847/2041-8205/825/2/L19/pdf), we define the effective binding energy:
$$
E_{\rm ISCO}^{\rm eff} = \sqrt{1 - \frac{2}{3r_{\rm ISCO}^{\rm eff}}}
$$

In [11]:
%%writefile -a $Ccodesdir/v5HM_BOB_Initial_Conditions_Calibration.txt
EISCOeff = np.sqrt(1 - 2/(3*rISCOeff))

Appending to Dynamics/v5HM_BOB_Initial_Conditions_Calibration.txt


# Step : $\mathcal{l}$

Following Equations 13 of [HBR2016](https://iopscience.iop.org/article/10.3847/2041-8205/825/2/L19/pdf), we define the angular momentum contribution as:
$$
\mathcal{l} = L_{\rm ISCO}^{\rm eff} - 2a_{\rm tot}(E_{\rm ISCO}^{\rm eff} - 1) + \sum_{i = 0}^{3}\sum_{j = 0}^{4} k_{ij}\nu^{2 + i}a^{\textrm{eff},j}
$$

Where, the NR coefficients $k_{ij}$ are given in Table 1 of HBR2016 for the choice of $n_M = 3, n_J = 4$.

$$
k_{00} = -5.97723\\
k_{01} = 3.39221\\
k_{02} = 4.48865\\
k_{03} = -5.77101\\
k_{04} = -13.0459\\
k_{10} = 35.1278\\
k_{11} = -72.9336\\
k_{12} = -86.0036\\
k_{13} = 93.7371\\
k_{14} = 200.975\\
k_{20} = -146.822\\
k_{21} = 387.184\\
k_{22} = 447.009\\
k_{23} = -467.383\\
k_{24} = -884.339\\
k_{30} = 223.911\\
k_{31} = -648.502\\
k_{32} = -697.177\\
k_{33} = 753.738\\
k_{34} = 1166.89
$$

In [12]:
%%writefile -a $Ccodesdir/v5HM_BOB_Initial_Conditions_Calibration.txt
k = np.zeros([4,5])
k[0,0] = -5.97723
k[0,1] = 3.39221
k[0,2] = 4.48865
k[0,3] = -5.77101
k[0,4] = -13.0459
k[1,0] = 35.1278
k[1,1] = -72.9336
k[1,2] = -86.0036
k[1,3] = 93.7371
k[1,4] = 200.975
k[2,0] = -146.822
k[2,1] = 387.184
k[2,2] = 447.009
k[2,3] = -467.383
k[2,4] = -884.339
k[3,0] = 223.911
k[3,1] = -648.502
k[3,2] = -697.177
k[3,3] = 753.738
k[3,4] = 1166.89

NRfactor = 0
nu_2plusi = nu*nu
for i in range(len(k)):
    aeff_j = 1 
    for j in range(len(k[i])):
        NRfactor += k[i,j]*nu_2plusi*aeff_j
        aeff_j *= aeff
    nu_2plusi *= nu
ell = np.abs(LISCOeff - 2*atot*(EISCOeff - 1) + NRfactor)

Appending to Dynamics/v5HM_BOB_Initial_Conditions_Calibration.txt


# Step : $a_{f}$

Following Equations 13 of [HBR2016](https://iopscience.iop.org/article/10.3847/2041-8205/825/2/L19/pdf), we define the final spin:
$$
a_{f} = a_{\rm tot} + \nu \mathcal{l}
$$

In [13]:
%%writefile -a $Ccodesdir/v5HM_BOB_Initial_Conditions_Calibration.txt
af = atot + nu*ell

Appending to Dynamics/v5HM_BOB_Initial_Conditions_Calibration.txt


# Step : $r_{\rm ISCO}$

Equations 4-6 of [HBR2016](https://iopscience.iop.org/article/10.3847/2041-8205/825/2/L19/pdf) are a general formula for the radius of the ISCO. In this case, we use the final spin calculated above:
$$
r_{\rm ISCO} = 3 + Z_2 - \frac{a_{f}}{|a_{f}|}\sqrt{(3 - Z_1)(3 + Z_1 + 2Z_2)}\\
Z_2 = \sqrt{3 a_{f}^{2} + Z_1^2}\\
Z_1 = 1 + (1 - a_{f}^{2})^{1/3}\left[ (1 + a_{f})^{1/3} + (1 - a_{f})^{1/3} \right]\\
$$

In [14]:
%%writefile -a $Ccodesdir/v5HM_BOB_Initial_Conditions_Calibration.txt
Z1f = 1 + (np.cbrt(1 - af*af))*(np.cbrt(1 + af) + np.cbrt(1 - af))
Z2f = np.sqrt(3*af*af + Z1f*Z1f)
rISCO = 3 + Z2f - np.sign(af)*np.sqrt((3 - Z1f)*(3 + Z1f + Z2f))

Appending to Dynamics/v5HM_BOB_Initial_Conditions_Calibration.txt


# Step : $r_{\rm stop}$

As defined earlier, the stopping radius is given by:

$$
r_{\rm stop} =
\bigg\lbrace
    \begin{array}{lr}
        0.98r_{\rm ISCO} & \Delta t_{\rm NR} > 0,\\
        -1 & \rm otherwise,
    \end{array}
$$

Where, $\Delta t$ is provided as an input parameter.

In [15]:
%%writefile -a $Ccodesdir/v5HM_BOB_Initial_Conditions_Calibration.txt
rstop = -1
if Deltat > 0:
    rstop = 0.98*rISCO

Appending to Dynamics/v5HM_BOB_Initial_Conditions_Calibration.txt


# Step : $M_{f}$

In accordance with pyseobnr, cf line 178 of [compute_hlms.py](https://git.ligo.org/waveforms/software/pyseobnr/-/blob/main/pyseobnr/eob/waveform/compute_hlms.py), [Jimenez-Forteza et. al](https://arxiv.org/pdf/1611.00332.pdf), hereby UIB2016, is used to compute the mass of the final black hole. From the discussion above Equation 4 and Equation 28 of UIB2016 we find:

$$
M_f = 1 - E_{\rm rad}\\
E_{\rm rad} \equiv E_{\rm rad}(\nu,\hat{S},\Delta\chi) = E_{\rm rad}(\nu,\hat{S}) + \Delta E_{\rm rad}(\eta,\hat{S},\Delta\chi) 
$$
Where, $E_{\rm rad}$ is the energy radiated through gravitational waves.
We derive all the quantities and build the final mass.

# Step : $\hat{S}$

We define $\hat{S}$ as per Equation 1 of [UIB2016](https://arxiv.org/pdf/1611.00332.pdf) as:

$$
\hat{S} = \frac{m_1^2\chi_1 + m_2^2\chi_2}{m_1^2 + m_2^2}
$$

In [16]:
%%writefile -a $Ccodesdir/v5HM_BOB_Initial_Conditions_Calibration.txt
Shat = (m1*m1*chi1 + m2*m2*chi2)/(m1*m1 + m2*m2)
Shat2 = Shat*Shat
Shat3 = Shat2*Shat
Shat4 = Shat3*Shat
nu2 = nu*nu
nu3 = nu2*nu
nu4 = nu3*nu
sqrt1m4nu = np.sqrt(1. - 4.*nu)

Appending to Dynamics/v5HM_BOB_Initial_Conditions_Calibration.txt


# Step : $\Delta \chi$

We define $\Delta \chi$ as per the discussion below Equation 1 of [UIB2016](https://arxiv.org/pdf/1611.00332.pdf):

$$
\Delta\chi = \chi_1-\chi_2
$$

In [17]:
%%writefile -a $Ccodesdir/v5HM_BOB_Initial_Conditions_Calibration.txt
Deltachi = chi1-chi2
Deltachi2 = Deltachi*Deltachi

Appending to Dynamics/v5HM_BOB_Initial_Conditions_Calibration.txt


# Step : $E_{\rm rad}(\nu,0)$

We define the non-spinning fit to the radiated energy as per Equation 21 of [UIB2016](https://arxiv.org/pdf/1611.00332.pdf):

$$
E_{\rm rad}(\nu,\hat{S} = 0) = a_4\nu^4 + a_3\nu^3 + a_2\nu^2 + \left(1 - \frac{2\sqrt{2}}{3}\right)\nu 
$$

Where, the coefficients are given in this [arXiv ancilliary file](https://arxiv.org/src/1611.00332v2/anc/FinalStateUIB2016_suppl_Erad_coeffs.txt):

$$
a_2 = 0.5609904135313374\\
a_3 = -0.84667563764404 \\
a_4 = 3.145145224278187 
$$

In [18]:
%%writefile -a $Ccodesdir/v5HM_BOB_Initial_Conditions_Calibration.txt
a2 = 0.5609904135313374 
a3 = -0.84667563764404
a4 = 3.145145224278187
Erad_nu_0 = a4*nu4 + a3*nu3 + a2*nu2 + (1. - 2.*np.sqrt(2.)/3.)*nu

Appending to Dynamics/v5HM_BOB_Initial_Conditions_Calibration.txt


# Step : $E_{\rm rad}(0.25,\hat{S})$ coefficients

Before implementing the spin contributions to the radiated energy, we should implement the modified coefficients. That is, the non-spinning and spinning contributions represent one dimensional fits to numerical relativity data at the higher end of the mass ratio. In order to develop a two dimensional fit in mass ratio and spin as well as to include effects at extreme mass ratio, coefficients for the spinning 1-D fit are modified (cf discussion above Equation 9 of [UIB2016](https://arxiv.org/pdf/1611.00332.pdf)).

The coefficients are defined in this [arXiv ancilliary file](https://arxiv.org/src/1611.00332v2/anc/FinalStateUIB2016_suppl_Erad_coeffs.txt).


$$
b_1 = -0.2091189048177395 \\
b_2 = -0.19709136361080587 \\
b_3 = -0.1588185739358418 \\
b_5 = 2.9852925538232014 \\
f_{20} = 4.271313308472851 \\
f_{30} = 31.08987570280556 \\
f_{50} = 1.5673498395263061 \\
f_{10} = 1.8083565298668276 \\
f_{21} = 0. \\
f_{11} = 15.738082204419655 \\
f_{31} = -243.6299258830685 \\
f_{51} = -0.5808669012986468 \\
f_{j2} = 16 - 16f_{j0} - 4f_{j1} \\
$$

The final coefficients given by Equation 9 of UIB2016:

$$
b^{\rm fin}_{i} = b_{i}\sum{j = 0}^{2} f_{ij}\nu^j
$$

In [19]:
%%writefile -a $Ccodesdir/v5HM_BOB_Initial_Conditions_Calibration.txt
b1 = -0.2091189048177395
b2 = -0.19709136361080587
b3 = -0.1588185739358418
b5 = 2.9852925538232014
f20 = 4.271313308472851
f30 = 31.08987570280556
f50 = 1.5673498395263061
f10 = 1.8083565298668276
f21 = 0.
d10 = -0.09803730445895877
d11 = -3.2283713377939134
d20 = 0.01118530335431078
d30 = -0.01978238971523653
d31 = -4.91667749015812
f11 = 15.738082204419655
f31 = -243.6299258830685
f51 = -0.5808669012986468
bfin1 = b1*(f10 + f11*nu + (16. - 16.*f10 - 4.*f11)*nu2)
bfin2 = b2*(f20 + f21*nu + (16. - 16.*f20 - 4.*f21)*nu2)
bfin3 = b3*(f30 + f31*nu + (16. - 16.*f30 - 4.*f31)*nu2)
bfin5 = b5*(f50 + f51*nu + (16. - 16.*f50 - 4.*f51)*nu2)

Appending to Dynamics/v5HM_BOB_Initial_Conditions_Calibration.txt


# Step : $E_{\rm rad}(0.25,\hat{S})$

We define the spin contributions fit to the radiated energy as per Equation 22 of [UIB2016](https://arxiv.org/pdf/1611.00332.pdf):

$$
E_{\rm rad}(\nu = 0.25,\hat{S}) = \frac{1 + 0.346b_1^{\rm fin}\hat{S} + 0.211b_2^{\rm fin}\hat{S}^2 + 0.128b_3^{\rm fin}\hat{S}^3}{1 - 0.212b_5^{\rm fin}\hat{S}} 
$$

Note: We do not use the multiplicative factor of 0.0484161 as it will be normalized away in the final fit (cf Equation 23 of UIB2016)

In [20]:
%%writefile -a $Ccodesdir/v5HM_BOB_Initial_Conditions_Calibration.txt
Erad_eq_Shat = (0.128*bfin3*Shat3 + 0.211*bfin2*Shat2 + 0.346*bfin1*Shat + 1.)/(1 - 0.212*bfin5*Shat)

Appending to Dynamics/v5HM_BOB_Initial_Conditions_Calibration.txt


# Step : $E_{\rm rad}(\nu,\hat{S})$

We define the 2-D fit to the radiated energy as per Equation 23 of [UIB2016](https://arxiv.org/pdf/1611.00332.pdf):

$$
E_{\rm rad}(\nu,\hat{S}) = E_{\rm rad}(\nu,0)E_{\rm rad}(0.25,\hat{S})
$$

In [21]:
%%writefile -a $Ccodesdir/v5HM_BOB_Initial_Conditions_Calibration.txt
Erad_nu_Shat = Erad_nu_0*Erad_eq_Shat

Appending to Dynamics/v5HM_BOB_Initial_Conditions_Calibration.txt


# Step : $\Delta E_{\rm rad}(\nu,\hat{S},\Delta\chi)$

We define the 3-D correction to the radiated energy as per Equation 27 of [UIB2016](https://arxiv.org/pdf/1611.00332.pdf) and the discussion above.:

$$
\Delta E_{\rm rad}(\nu,\hat{S},\Delta \chi) = A_1\Delta \chi + A_2 \Delta\chi^2 + A_3\hat{S}\Delta\chi\\
A_1 = d_{10}\sqrt{1 - 4\nu}\nu^2(d_{11}\nu + 1) \\
A_2 = d_{20}\nu^3 \\
A_3 = d_{30}\sqrt{1 - 4\nu}\nu(d_{31}\nu + 1) \\
$$

The coefficients are given in this [arXiv ancilliary file](https://arxiv.org/src/1611.00332v2/anc/FinalStateUIB2016_suppl_Erad_coeffs.txt):

$$
d_{10} = -0.09803730445895877 \\
d_{11} = -3.2283713377939134 \\
d_{20} = 0.01118530335431078 \\
d_{30} = -0.01978238971523653 \\
d_{31} = -4.91667749015812 \\
$$

In [22]:
%%writefile -a $Ccodesdir/v5HM_BOB_Initial_Conditions_Calibration.txt
d10 = -0.09803730445895877
d11 = -3.2283713377939134
d20 = 0.01118530335431078
d30 = -0.01978238971523653
d31 = -4.91667749015812
A_1 = d10*sqrt1m4nu*nu2*(d11*nu+1)
A_2 = d20*nu3
A_3 = d30*sqrt1m4nu*nu*(d31*nu + 1)
DeltaErad_nu_Shat_Deltachi = A_1*Deltachi + A_2*Deltachi2 + A_3*Deltachi*Shat

Appending to Dynamics/v5HM_BOB_Initial_Conditions_Calibration.txt


# Step : $M_f$

We define the full fit to the radiated energy as per Equation 28 of [UIB2016](https://arxiv.org/pdf/1611.00332.pdf) and thus, the final mass as per Equation 4 of the same:

$$
E_{\rm rad}(\nu,\hat{S},\Delta\chi) = E_{\rm rad}(\nu,\hat{S}) + \Delta E_{\rm rad}(\nu,\hat{S},\Delta\chi)\\
M_f = 1 - E_{\rm rad}(\nu,\hat{S},\Delta \chi)
$$

In [23]:
%%writefile -a $Ccodesdir/v5HM_BOB_Initial_Conditions_Calibration.txt
Mf = 1 - (Erad_nu_Shat + DeltaErad_nu_Shat_Deltachi)

Appending to Dynamics/v5HM_BOB_Initial_Conditions_Calibration.txt


# Step : Strain Amplitude

We define the numerical fit to the peak strain amplitude of the $l = 2, m = 2$ mode as per Equation C8 of [UIB2016](https://arxiv.org/pdf/1611.00332.pdf) and thus, the final mass as per Equation 4 of the same:

$$
h_{22}^{\rm NR} = \nu\left|71.97969776036882194603 \nu^4
            - 13.35761402231352157344 \nu^3 \hat{S}
            - 46.87585958426210908101 \nu^3
            + 0.61988944517825661507 \nu^2 \hat{S}^2
            + 7.19426416189229733789 \nu^2 \hat{S}
            + 12.44040490932310127903 \nu^2
            + 0.43014673069078152023 \nu \hat{S}^3
            - 1.74313546783413597652 \nu \hat{S}
            - 0.86828935763242798274 \nu
            - 0.08493901280736430859 \hat{S}^3
            - 0.02082621429567295401 \hat{S}^2
            + 0.18693991146784910695 \hat{S}
            + 1.46709663479911811557
\right|
$$

In [24]:
%%writefile -a $Ccodesdir/v5HM_BOB_Initial_Conditions_Calibration.txt
h22NR = nu*np.abs(71.97969776036882194603 * nu4
            - 13.35761402231352157344 * nu3 * Shat
            - 46.87585958426210908101 * nu3
            + 0.61988944517825661507 * nu2 * Shat2
            + 7.19426416189229733789 * nu2 * Shat
            + 12.44040490932310127903 * nu2
            + 0.43014673069078152023 * nu * Shat3
            - 1.74313546783413597652 * nu * Shat
            - 0.86828935763242798274 * nu
            - 0.08493901280736430859 * Shat3
            - 0.02082621429567295401 * Shat2
            + 0.18693991146784910695 * Shat
            + 1.46709663479911811557)

Appending to Dynamics/v5HM_BOB_Initial_Conditions_Calibration.txt


# Step : Frequency

We define the numerical fit to the waveform frequency corresponding to the peak strain amplitude as per Equation C29 of [UIB2016](https://arxiv.org/pdf/1611.00332.pdf):

$$
\omega_{22}^{\rm NR} = \left(5.89352329617707670906 \nu^4
            + 3.75145580491965446868 \nu^3 \hat{S}
            - 3.34930536209472551334 \nu^3
            - 0.97140932625194231775 \nu^2 \hat{S}^2
            - 1.69734302394369973577 \nu^2 \hat{S}
            + 0.28539204856044564362 \nu^2
            + 0.2419483723662931296 \nu\hat{S}^3
            + 0.51801427018052081941 \nu\hat{S}^2
            + 0.25096450064948544467 \nu\hat{S}
            - 0.31709602351033533418 \nu
            - 0.01525897668158244028 \hat{S}^4
            - 0.06692658483513916345 \hat{S}^3
            - 0.08715176045684569495 \hat{S}^2
            - 0.09133931944098934441 \hat{S}
            - 0.2685414392185025978
\right)
$$

In [25]:
%%writefile -a $Ccodesdir/v5HM_BOB_Initial_Conditions_Calibration.txt
omega22NR = (5.89352329617707670906 * nu4
            + 3.75145580491965446868 * nu3 * Shat
            - 3.34930536209472551334 * nu3
            - 0.97140932625194231775 * nu2 * Shat2
            - 1.69734302394369973577 * nu2 * Shat
            + 0.28539204856044564362 * nu2
            + 0.2419483723662931296 * nu * Shat3
            + 0.51801427018052081941 * nu * Shat2
            + 0.25096450064948544467 * nu * Shat
            - 0.31709602351033533418 * nu
            - 0.01525897668158244028 * Shat4
            - 0.06692658483513916345 * Shat3
            - 0.08715176045684569495 * Shat2
            - 0.09133931944098934441 * Shat
            - 0.2685414392185025978
)

Appending to Dynamics/v5HM_BOB_Initial_Conditions_Calibration.txt


# Step : Store the routine in a python file

In [26]:
with open(os.path.join(Ccodesdir,"v5HM_BOB_initial_conditions_calibration.py"),"w") as output:
    output.write("import numpy as np\nfrom scipy.optimize import root, root_scalar\nfrom Derivatives.v5HM_BOB_initial_conditions_conservative_calibration import v5HM_BOB_unoptimized_IC_cons_calibration as IC_cons\nfrom Derivatives.v5HM_BOB_initial_conditions_dissipative_calibration import v5HM_BOB_unoptimized_IC_diss_calibration as IC_diss\n")
    output.write("def v5HM_BOB_unoptimized_initial_conditions_calibration(M,q,S1,S2,f,a6,dSO,Deltat):\n")
    for line in list(open(os.path.join(Ccodesdir,"v5HM_BOB_Initial_Conditions_Calibration.txt"),"r")):
        output.write("    %s \n"%line.rstrip())
    output.write("    return m1,m2,chi1,chi2,np.array([r,0.,prstar,pphi]),Omega_0,h,rstop,rISCO,af,Mf,h22NR,omega22NR")
    
import numpy as np
from Dynamics.v5HM_Initial_Conditions import v5HM_unoptimized_initial_conditions as IC_final
from Dynamics.pyseobnr_initial_conditions import computeIC_opt as IC_true
M = 33.
q = 23./10.
f = 20
chi_1 = 0.5
chi_2 = 0.9
Msol = 4.925491025543575903411922162094833998e-6
Omega_start = M*Msol*np.pi*f
m1 = q/(1 + q)
m2 = 1/(1 + q)
print(IC_final(M,q,chi_1,chi_2,f),IC_true(Omega_start,chi_1, chi_2, m1, m2))

(0.6969696969696969, 0.30303030303030304, 0.5, 0.9, array([ 2.12155008e+01,  0.00000000e+00, -2.86537636e-04,  4.88471098e+00]), 0.010212765037972303, 123.04572334363787, 2.2236685572617088, 2.2690495482262336, 6.493255996236028, 0.9651759994518843, 0.9461645795886238, 0.3276331834048432, -0.3980351614903812) [21.21550081935077, 0.0, -0.00028653763604539854, 4.884710982418403]
